New attempt to make it better.

In [1]:
%matplotlib inline

In [2]:
import numpy as np
from pandas import *
from ggplot import *

In [5]:
turnstile_weather = pandas.read_csv('data/turnstile_data_master_with_weather.csv')

In [6]:
print turnstile_weather.columns

Index([u'Unnamed: 0', u'UNIT', u'DATEn', u'TIMEn', u'Hour', u'DESCn',
       u'ENTRIESn_hourly', u'EXITSn_hourly', u'maxpressurei', u'maxdewpti',
       u'mindewpti', u'minpressurei', u'meandewpti', u'meanpressurei', u'fog',
       u'rain', u'meanwindspdi', u'mintempi', u'meantempi', u'maxtempi',
       u'precipi', u'thunder'],
      dtype='object')


In [7]:
def normalize_features(df):
    mu = df.mean()
    sigma = df.std()    
    if (sigma == 0).any():
        raise Exception("One or more features had the same value for all samples, and thus could " + \
                         "not be normalized. Please do not include features with only a single value " + \
                         "in your model.")
    df_normalized = (df - df.mean()) / df.std()
    return df_normalized, mu, sigma

In [8]:
def compute_cost(features, values, theta):
    m = len(values)
    sum_of_square_errors = np.square(np.dot(features, theta) - values).sum()
    cost = sum_of_square_errors / (2*m)
    return cost

In [9]:
def gradient_descent(features, values, theta, alpha, num_iterations):
    m = len(values)
    cost_history = []
    for i in range(num_iterations):
        predicted_values = np.dot(features, theta)
        theta = theta - alpha / m * np.dot((predicted_values - values), features)
        cost = compute_cost(features, values, theta)
        cost_history.append(cost)       
    return theta, pandas.Series(cost_history)

In [11]:
def compute_r_squared(data, predictions):
    r_squared = 1 - np.square(data - predictions).sum() / np.square(data - data.mean()).sum()
    return r_squared

In [15]:
features = turnstile_weather[['rain', 'precipi', 'Hour', 'mintempi', 'fog']]
dummy_units = pandas.get_dummies(turnstile_weather['UNIT'], prefix='unit')
features = features.join(dummy_units)
values = turnstile_weather['ENTRIESn_hourly']
m = len(values)
features, mu, sigma = normalize_features(features)
features['ones'] = np.ones(m) # Add a column of 1s (y intercept)
features_array = np.array(features)
values_array = np.array(values)    

In [17]:
alpha = 0.1 # please feel free to change this value
num_iterations = 75 # please feel free to change this value
theta_gradient_descent = np.zeros(len(features.columns))
theta_gradient_descent, cost_history = gradient_descent(features_array, 
                                                        values_array, 
                                                        theta_gradient_descent, 
                                                        alpha, 
                                                        num_iterations)
prediction = np.dot(features_array, theta_gradient_descent)

In [27]:
print theta_gradient_descent

[ -1.33673022e+01   2.82801860e+00   4.64131479e+02  -7.12795553e+01
   4.20040010e+01   1.08906479e+02  -5.92190069e+00  -3.09481983e+01
  -1.93952483e+01  -1.98079812e+01  -1.78887331e+01  -2.48924860e+01
  -2.32449620e+01  -2.63787794e+01   1.30021822e+02   2.69675486e+02
   2.55387822e+02   5.48523409e+01   1.12241509e+02   4.57453924e+01
  -3.46143535e+00   1.20334547e+02   2.04283263e+02   7.20396845e+01
   2.15902582e+02   1.34999308e+02   3.00691453e+02   2.10754349e+02
   7.49032511e+01   1.56903104e+02   7.33151149e+01   4.86146194e+01
   2.13127577e+02   6.85622479e+01   1.33246957e+02   1.17463396e+02
   2.89606531e+02  -1.75016818e-03   8.37247067e+01  -1.00170939e+01
  -7.36378872e+00  -3.04231911e+01  -8.75897667e+00   5.94112722e+00
   6.61363118e+01  -1.43159947e+01  -3.29736738e+01   1.22393932e+02
   6.86883584e+01   2.79453711e+02   1.48276059e+02   7.99814267e+00
   7.30611042e+01   1.05433584e+02   1.50851825e+02   6.65404603e+00
   8.08610228e+01   1.30997862e+01

In [18]:
print compute_r_squared(values_array, prediction)

0.458583922399


In [22]:
import statsmodels.api as sm

In [23]:
features = turnstile_weather[['rain', 'precipi', 'Hour', 'mintempi', 'fog']]
values = turnstile_weather['ENTRIESn_hourly']
features_array = np.array(features)
values_array = np.array(values)

In [24]:
features_array_constant = sm.add_constant(features_array)
model = sm.OLS(values_array, features_array_constant)
theta_statsmodels = model.fit().params
prediction = np.dot(features_array_constant, theta_statsmodels)    

In [26]:
print theta_statsmodels

[ 1106.72868003   -34.09989023     8.51130258    59.48934704   -11.92516896
   120.76571078]


In [25]:
print compute_r_squared(values_array, prediction)

0.0319297936738
